In [ ]:
# The code below imports pandas and numpy libraries for data analysis and manipulation



In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import json



# Load dataset
df = pd.read_csv('H:/datasets/games.csv')

# Visualize dataframe
df.head()


In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-73996d74-b78d-4ba2-bb4f-7f7f82c8b46d"
}


# auxiliary function
def determine_overall_region(region):
    overall_region = str()
    tagline = str()
    if region in ['euw1', 'eun1', 'ru', 'tr1']:
        overall_region = 'europe'
    elif region in ['br1', 'la1', 'la2', 'na1']:
        overall_region = 'americas'
    else:
        overall_region = 'asia'
    # BR1, EUNE, EUW, JP1, KR, LA1, LA2, NA, OCE, RU, TR
    if region in ['br1', 'jp1', 'kr', 'la1', 'la2', 'ru', 'na1', 'tr1', 'oc1']:
        tagline = region.upper()
    elif region == 'euw1':
        tagline = 'EUW'
    elif region == 'eun1':
        tagline = 'EUNE'
    # 3 cases left: OCE
    return overall_region, tagline


def get_match_info(match_id, region):
    available_regions = ['europe', 'americas', 'asia']
    assert region in available_regions
    request_url = 'https://{}.api.riotgames.com/lol/match/v5/matches/{}'.format(
        region,
        match_id
    )
    print(match_id)


    response = requests.get(request_url, headers=headers)

    #print(json.dumps(response.json()))
    if response.status_code == 200:
        print('{} {}'.format(time.strftime("%Y-%m-%d %H:%M"), response.json()))
    else:
        print('{} Request error (@get_match_info). HTTP code {}'.format(time.strftime("%Y-%m-%d %H:%M"), response.status_code))
    # Return the list of matches.
    # Write the json to a .txt file
    with open('example.txt', 'w') as f:
        json.dump(response.json(), f)
    return response.json()


def process_player_performance(obj):
    #obj['info']['gameDuration'] # milliseconds
    #obj['participants'][0] -> everything
    # create new dataset from this.

    # flatten json object to csv format

    #obj['participants'][0]['win'] -> to predict

    #and also, to predict player performance -> or take as input for now.

    for i in range(len(obj['info']['participants'])):


        duration_s = obj['info']['gameDuration'] / 1000
        duration_m = duration_s / 60
        match_identifier = obj['info']['gameId']


        deaths_per_min = obj['info']['participants'][i]['deaths'] / duration_m
        k_a_per_min = (obj['info']['participants'][i]['kills'] + obj['info']['participants'][i]['assists']) / duration_m
        level_per_min = obj['info']['participants'][i]['champLevel'] / duration_m
        total_damage_per_min = obj['info']['participants'][i]['totalDamageDealt'] / duration_m
        gold_per_min = obj['info']['participants'][i]['goldEarned'] / duration_m

        # 0.336 — (1.437 x Deaths per min) + (0.000117 x gold per min) + (0.443 x K_A per min) + (0.264 x Level per min) + (0.000013 x TD per min)
        calculated_player_performance = 0.336 - (1.437*deaths_per_min) + (0.000117*gold_per_min) + (0.443*k_a_per_min) + (0.264 * level_per_min) + (0.000013 * total_damage_per_min)
        new_object = {
            'match_identifier': match_identifier,
            'duration': duration_m,
            'f1': deaths_per_min,
            'f2': k_a_per_min,
            'f3': level_per_min,
            'f4': total_damage_per_min,
            'f5': gold_per_min,
            'calculated_player_performance': round((calculated_player_performance * 100), 2)
        }
        final_object = obj['info']['participants'][i] | new_object # merge two dictionaries
        print('{}: {}'.format(round((calculated_player_performance * 100), 2), json.dumps(final_object)))
        # Use pandas to convert the final_object to a DataFrame and then to a tabular structure
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(final_object)
        #print(df)
        # Use pandas to convert the final_object to a DataFrame and then to a tabular structure

        # Write the DataFrame to a CSV file in tabular format
        df.to_csv('processed_performance.csv', index=False, header=False, mode='a')  # Set index=False to exclude the index column from the output file. append mode.




# Iterate over df
for row, index in df.iterrows():
    match_id = row
    #print(match_id)
    #print(determine_overall_region(str(match_id)[0:3].lower())[0])
    results = get_match_info(match_id, determine_overall_region(str(match_id)[0:3].lower())[0])

    process_player_performance(results)

    time.sleep(1)

